In [2]:
from __future__ import print_function
import random
import os
import sys
import copy
import math
import time
import pickle
import contextlib
import numpy as np
import pandas as pd
import multiprocessing as mp
from deap import tools
from deap import base, creator
from sklearn.cluster import KMeans
from multiprocessing import Pool
from six.moves.urllib.request import urlretrieve
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Input, Dense, Activation,AlphaDropout
from keras.optimizers import adam, SGD
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\envs\deeplearning\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
data_root = 'data'
german_data = os.path.join(data_root,'GermanCreditInput.xls')
german_label = os.path.join(data_root,'GermanCreditOutputClass1columnknn.xls')
australian_dataset = os.path.join(data_root,'australian dataset.xlsx')

In [4]:
nn_config = {
    'train_cycles':600,
}

In [5]:
x_data = np.array(pd.read_excel(german_data,header=None))
y_data = np.array(pd.read_excel(german_label,header=None))
print('Dataset Shape : {}\nDataset Labels Shape : {}'.format(x_data.shape,y_data.shape))

Dataset Shape : (1000, 30)
Dataset Labels Shape : (1000, 1)


In [6]:
mx = MinMaxScaler()
mx.fit(x_data)
p_x_data = mx.transform(x_data)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\envs\deeplearning\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [13]:
model = Sequential()
model.add(Dense(100, input_dim=30, kernel_initializer='glorot_normal', activation='selu'))
model.add(AlphaDropout(.2))
model.add(Dense(75, kernel_initializer='glorot_normal', activation='selu'))
model.add(Dense(50, kernel_initializer='glorot_normal', activation='selu'))
model.add(AlphaDropout(.2))
model.add(Dense(25,  kernel_initializer='glorot_normal', activation='selu'))
model.add(Dense(1, kernel_initializer='glorot_normal', activation='sigmoid'))
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer='rmsprop')
skf = StratifiedKFold(n_splits=10)
for train_index, test_index in skf.split(p_x_data, y_data):
    model.fit_generator()
    model.fit(mx.transform(x_data)[train_index,:],y_data[train_index,:], epochs=int(600),batch_size=16,verbose=1,validation_data=(mx.transform(x_data)[test_index,:],y_data[test_index,:]))
    break;

Train on 900 samples, validate on 100 samples
Epoch 1/600
900/900 [==============================] - 2s 2ms/step - loss: 0.7289 - acc: 0.6422 - val_loss: 0.5541 - val_acc: 0.7000
Epoch 2/600
900/900 [==============================] - 1s 669us/step - loss: 0.6188 - acc: 0.6956 - val_loss: 0.6689 - val_acc: 0.7000
Epoch 3/600
900/900 [==============================] - 1s 793us/step - loss: 0.5771 - acc: 0.7067 - val_loss: 0.6364 - val_acc: 0.7100
Epoch 4/600
900/900 [==============================] - 1s 776us/step - loss: 0.5840 - acc: 0.7144 - val_loss: 0.5931 - val_acc: 0.7300
Epoch 5/600
900/900 [==============================] - 1s 789us/step - loss: 0.5663 - acc: 0.7178 - val_loss: 0.5025 - val_acc: 0.7600
Epoch 6/600
900/900 [==============================] - 1s 819us/step - loss: 0.5591 - acc: 0.7200 - val_loss: 0.5805 - val_acc: 0.7400
Epoch 7/600
900/900 [==============================] - 1s 845us/step - loss: 0.5412 - acc: 0.7200 - val_loss: 0.6264 - val_acc: 0.7400
Epoch 8/600

Epoch 61/600
900/900 [==============================] - 1s 781us/step - loss: 0.4678 - acc: 0.7644 - val_loss: 0.5437 - val_acc: 0.7800
Epoch 62/600
900/900 [==============================] - 1s 766us/step - loss: 0.4782 - acc: 0.7711 - val_loss: 0.4682 - val_acc: 0.7800
Epoch 63/600
900/900 [==============================] - 1s 739us/step - loss: 0.4666 - acc: 0.7667 - val_loss: 0.4609 - val_acc: 0.7800
Epoch 64/600
900/900 [==============================] - 1s 769us/step - loss: 0.4697 - acc: 0.7711 - val_loss: 0.4537 - val_acc: 0.7800
Epoch 65/600
900/900 [==============================] - 1s 750us/step - loss: 0.4624 - acc: 0.7744 - val_loss: 0.4730 - val_acc: 0.8000
Epoch 66/600
900/900 [==============================] - 1s 711us/step - loss: 0.4650 - acc: 0.7733 - val_loss: 0.4868 - val_acc: 0.7600
Epoch 67/600
900/900 [==============================] - 1s 769us/step - loss: 0.4523 - acc: 0.7922 - val_loss: 0.4738 - val_acc: 0.7900
Epoch 68/600
900/900 [==========================

900/900 [==============================] - 1s 769us/step - loss: 0.4444 - acc: 0.7967 - val_loss: 0.4472 - val_acc: 0.7900
Epoch 122/600
900/900 [==============================] - 1s 781us/step - loss: 0.4386 - acc: 0.7878 - val_loss: 0.4829 - val_acc: 0.8000
Epoch 123/600
900/900 [==============================] - 1s 726us/step - loss: 0.4419 - acc: 0.7900 - val_loss: 0.4805 - val_acc: 0.8000
Epoch 124/600
900/900 [==============================] - 1s 715us/step - loss: 0.4363 - acc: 0.8011 - val_loss: 0.4621 - val_acc: 0.8200
Epoch 125/600
900/900 [==============================] - 1s 771us/step - loss: 0.4424 - acc: 0.7933 - val_loss: 0.5001 - val_acc: 0.8200
Epoch 126/600
900/900 [==============================] - 1s 780us/step - loss: 0.4433 - acc: 0.7978 - val_loss: 0.4774 - val_acc: 0.8100
Epoch 127/600
900/900 [==============================] - 1s 716us/step - loss: 0.4263 - acc: 0.7989 - val_loss: 0.4283 - val_acc: 0.8200
Epoch 128/600
900/900 [==============================] 

Epoch 181/600
900/900 [==============================] - 1s 777us/step - loss: 0.3836 - acc: 0.8278 - val_loss: 0.5318 - val_acc: 0.8100
Epoch 182/600
900/900 [==============================] - 1s 761us/step - loss: 0.3821 - acc: 0.8378 - val_loss: 0.4681 - val_acc: 0.8300
Epoch 183/600
900/900 [==============================] - 1s 717us/step - loss: 0.3875 - acc: 0.8244 - val_loss: 0.5053 - val_acc: 0.8200
Epoch 184/600
900/900 [==============================] - 1s 737us/step - loss: 0.3869 - acc: 0.8244 - val_loss: 0.5325 - val_acc: 0.8100
Epoch 185/600
900/900 [==============================] - 1s 769us/step - loss: 0.3851 - acc: 0.8289 - val_loss: 0.5236 - val_acc: 0.8100
Epoch 186/600
900/900 [==============================] - 1s 778us/step - loss: 0.3924 - acc: 0.8167 - val_loss: 0.5201 - val_acc: 0.8000
Epoch 187/600
900/900 [==============================] - 1s 826us/step - loss: 0.3815 - acc: 0.8167 - val_loss: 0.5279 - val_acc: 0.7600
Epoch 188/600
900/900 [==================

Epoch 241/600
900/900 [==============================] - 1s 767us/step - loss: 0.3269 - acc: 0.8644 - val_loss: 0.6379 - val_acc: 0.7800
Epoch 242/600
900/900 [==============================] - 1s 760us/step - loss: 0.3365 - acc: 0.8389 - val_loss: 0.6168 - val_acc: 0.8100
Epoch 243/600
900/900 [==============================] - 1s 756us/step - loss: 0.3260 - acc: 0.8467 - val_loss: 0.5953 - val_acc: 0.8000
Epoch 244/600
900/900 [==============================] - 1s 755us/step - loss: 0.3510 - acc: 0.8533 - val_loss: 0.5958 - val_acc: 0.7900
Epoch 245/600
900/900 [==============================] - 1s 759us/step - loss: 0.3201 - acc: 0.8567 - val_loss: 0.6290 - val_acc: 0.8100
Epoch 246/600
900/900 [==============================] - 1s 720us/step - loss: 0.3225 - acc: 0.8522 - val_loss: 0.5721 - val_acc: 0.8000
Epoch 247/600
900/900 [==============================] - 1s 698us/step - loss: 0.3332 - acc: 0.8444 - val_loss: 0.6195 - val_acc: 0.7900
Epoch 248/600
900/900 [==================

Epoch 301/600
900/900 [==============================] - 1s 699us/step - loss: 0.2716 - acc: 0.8911 - val_loss: 0.7192 - val_acc: 0.8100
Epoch 302/600
900/900 [==============================] - 1s 706us/step - loss: 0.2826 - acc: 0.8733 - val_loss: 0.7098 - val_acc: 0.8200
Epoch 303/600
900/900 [==============================] - 1s 759us/step - loss: 0.2705 - acc: 0.8867 - val_loss: 0.7048 - val_acc: 0.7900
Epoch 304/600
900/900 [==============================] - 1s 739us/step - loss: 0.2810 - acc: 0.8833 - val_loss: 0.7516 - val_acc: 0.7900
Epoch 305/600
900/900 [==============================] - 1s 769us/step - loss: 0.2760 - acc: 0.8922 - val_loss: 0.8098 - val_acc: 0.7900
Epoch 306/600
900/900 [==============================] - 1s 752us/step - loss: 0.2588 - acc: 0.8922 - val_loss: 0.7699 - val_acc: 0.8000
Epoch 307/600
900/900 [==============================] - 1s 725us/step - loss: 0.2855 - acc: 0.8622 - val_loss: 0.7445 - val_acc: 0.7900
Epoch 308/600
900/900 [==================

Epoch 361/600
900/900 [==============================] - 1s 561us/step - loss: 0.2502 - acc: 0.8944 - val_loss: 0.7395 - val_acc: 0.8000
Epoch 362/600
900/900 [==============================] - 1s 625us/step - loss: 0.2160 - acc: 0.9111 - val_loss: 0.7435 - val_acc: 0.8300
Epoch 363/600
900/900 [==============================] - 1s 676us/step - loss: 0.2211 - acc: 0.9044 - val_loss: 0.8075 - val_acc: 0.7900
Epoch 364/600
900/900 [==============================] - 1s 666us/step - loss: 0.2465 - acc: 0.9056 - val_loss: 0.7849 - val_acc: 0.7800
Epoch 365/600
900/900 [==============================] - 1s 687us/step - loss: 0.2289 - acc: 0.9111 - val_loss: 0.8232 - val_acc: 0.7800
Epoch 366/600
900/900 [==============================] - 1s 688us/step - loss: 0.2353 - acc: 0.9022 - val_loss: 0.8352 - val_acc: 0.7900
Epoch 367/600
900/900 [==============================] - 1s 703us/step - loss: 0.2208 - acc: 0.9144 - val_loss: 0.8501 - val_acc: 0.8200
Epoch 368/600
900/900 [==================

Epoch 421/600
900/900 [==============================] - 1s 860us/step - loss: 0.2145 - acc: 0.9156 - val_loss: 0.8515 - val_acc: 0.8000
Epoch 422/600
900/900 [==============================] - 1s 824us/step - loss: 0.1906 - acc: 0.9211 - val_loss: 0.9274 - val_acc: 0.8000
Epoch 423/600
900/900 [==============================] - 1s 766us/step - loss: 0.1861 - acc: 0.9178 - val_loss: 0.9521 - val_acc: 0.8000
Epoch 424/600
900/900 [==============================] - 1s 830us/step - loss: 0.1812 - acc: 0.9222 - val_loss: 1.0265 - val_acc: 0.7900
Epoch 425/600
900/900 [==============================] - 1s 572us/step - loss: 0.2134 - acc: 0.9189 - val_loss: 0.9131 - val_acc: 0.7700
Epoch 426/600
900/900 [==============================] - 1s 633us/step - loss: 0.1999 - acc: 0.9178 - val_loss: 1.0539 - val_acc: 0.7500
Epoch 427/600
900/900 [==============================] - 1s 701us/step - loss: 0.2073 - acc: 0.9089 - val_loss: 0.9256 - val_acc: 0.8000
Epoch 428/600
900/900 [==================

Epoch 481/600
900/900 [==============================] - 1s 681us/step - loss: 0.1637 - acc: 0.9322 - val_loss: 1.0709 - val_acc: 0.7900
Epoch 482/600
900/900 [==============================] - 1s 767us/step - loss: 0.1586 - acc: 0.9322 - val_loss: 1.1511 - val_acc: 0.7700
Epoch 483/600
900/900 [==============================] - 1s 740us/step - loss: 0.1833 - acc: 0.9244 - val_loss: 1.1191 - val_acc: 0.7800
Epoch 484/600
900/900 [==============================] - 1s 675us/step - loss: 0.1648 - acc: 0.9356 - val_loss: 0.9853 - val_acc: 0.8000
Epoch 485/600
900/900 [==============================] - 1s 667us/step - loss: 0.1896 - acc: 0.9189 - val_loss: 1.0265 - val_acc: 0.7900
Epoch 486/600
900/900 [==============================] - 1s 688us/step - loss: 0.1664 - acc: 0.9333 - val_loss: 1.0791 - val_acc: 0.7800
Epoch 487/600
900/900 [==============================] - 1s 671us/step - loss: 0.1602 - acc: 0.9244 - val_loss: 1.3565 - val_acc: 0.7800
Epoch 488/600
900/900 [==================

Epoch 541/600
900/900 [==============================] - 1s 694us/step - loss: 0.1597 - acc: 0.9333 - val_loss: 1.2051 - val_acc: 0.8000
Epoch 542/600
900/900 [==============================] - 1s 747us/step - loss: 0.1599 - acc: 0.9356 - val_loss: 1.1983 - val_acc: 0.7800
Epoch 543/600
900/900 [==============================] - 1s 775us/step - loss: 0.1667 - acc: 0.9333 - val_loss: 1.2174 - val_acc: 0.7700
Epoch 544/600
900/900 [==============================] - 1s 797us/step - loss: 0.1408 - acc: 0.9422 - val_loss: 1.2428 - val_acc: 0.7700
Epoch 545/600
900/900 [==============================] - 1s 649us/step - loss: 0.1814 - acc: 0.9311 - val_loss: 1.1373 - val_acc: 0.7800
Epoch 546/600
900/900 [==============================] - 1s 776us/step - loss: 0.1612 - acc: 0.9456 - val_loss: 1.1053 - val_acc: 0.7900
Epoch 547/600
900/900 [==============================] - 1s 776us/step - loss: 0.1303 - acc: 0.9478 - val_loss: 1.3555 - val_acc: 0.7800
Epoch 548/600
900/900 [==================

In [11]:
sum_val_acc = 0
skf = StratifiedKFold(n_splits=10)
sgd = SGD(lr=0.3, momentum=0.7)
for train_index, test_index in skf.split(x_data, y_data):
    model = Sequential()
    model.add(Dense(60, input_dim=30, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer='rmsprop')
    hist = model.fit_generator(data_generator(p_x_data[train_index,:],y_data[train_index,:],32),epochs=int(10),steps_per_epoch=40,validation_data=(mx.transform(x_data)[test_index,:],y_data[test_index,:]))
    x = model.evaluate(mx.transform(x_data)[test_index,:],y_data[test_index,:])
    print(x)
#     hist = model.fit(mx.transform(x_data)[train_index,:],y_data[train_index,:], epochs=int(400),batch_size=len(train_index),verbose=0,validation_data=(mx.transform(x_data)[test_index,:],y_data[test_index,:]))
    sum_val_acc += hist.history['val_acc'].pop()
    del model
print(sum_val_acc/10)

Epoch 1/10
40/40 [==============================] - 1s 15ms/step - loss: 0.7135 - acc: 0.5414 - val_loss: 0.6956 - val_acc: 0.5000
Epoch 2/10
40/40 [==============================] - 0s 4ms/step - loss: 0.6624 - acc: 0.6539 - val_loss: 0.6557 - val_acc: 0.6200
Epoch 3/10
40/40 [==============================] - 0s 3ms/step - loss: 0.6340 - acc: 0.7367 - val_loss: 0.6213 - val_acc: 0.6800
Epoch 4/10
40/40 [==============================] - 0s 4ms/step - loss: 0.6299 - acc: 0.7086 - val_loss: 0.6143 - val_acc: 0.6700
Epoch 5/10
40/40 [==============================] - 0s 3ms/step - loss: 0.6148 - acc: 0.7141 - val_loss: 0.6185 - val_acc: 0.6700
Epoch 6/10
40/40 [==============================] - 0s 3ms/step - loss: 0.6042 - acc: 0.7016 - val_loss: 0.5856 - val_acc: 0.6900
Epoch 7/10
40/40 [==============================] - 0s 3ms/step - loss: 0.5871 - acc: 0.7359 - val_loss: 0.5988 - val_acc: 0.7000
Epoch 8/10
40/40 [==============================] - 0s 3ms/step - loss: 0.5832 - acc: 0.7

100/100 [==============================] - 0s 73us/step
[0.5993511080741882, 0.64]
Epoch 1/10
40/40 [==============================] - 0s 12ms/step - loss: 0.6826 - acc: 0.5703 - val_loss: 0.6752 - val_acc: 0.6400
Epoch 2/10
40/40 [==============================] - 0s 4ms/step - loss: 0.6554 - acc: 0.6945 - val_loss: 0.6743 - val_acc: 0.6000
Epoch 3/10
40/40 [==============================] - 0s 3ms/step - loss: 0.6348 - acc: 0.6984 - val_loss: 0.6446 - val_acc: 0.6800
Epoch 4/10
40/40 [==============================] - 0s 3ms/step - loss: 0.6239 - acc: 0.7133 - val_loss: 0.6364 - val_acc: 0.6900
Epoch 5/10
40/40 [==============================] - 0s 3ms/step - loss: 0.6111 - acc: 0.6992 - val_loss: 0.6255 - val_acc: 0.6800
Epoch 6/10
40/40 [==============================] - 0s 3ms/step - loss: 0.5909 - acc: 0.7266 - val_loss: 0.6606 - val_acc: 0.6000
Epoch 7/10
40/40 [==============================] - 0s 3ms/step - loss: 0.5700 - acc: 0.7406 - val_loss: 0.6099 - val_acc: 0.6800
Epoch 

In [44]:
for i in data_generator(p_x_data,y_data,10) :
    print(i)
    break

(array([[1.        , 0.11764706, 0.5       , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.07285133,
        0.        , 0.5       , 1.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.33333333, 1.        ,
        0.        , 0.55357143, 0.        , 0.        , 1.        ,
        0.        , 0.66666667, 0.        , 0.        , 0.        ],
       [1.        , 0.11764706, 1.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.11450424,
        1.        , 1.        , 0.        , 0.        , 1.        ,
        0.        , 1.        , 0.        , 1.        , 1.        ,
        0.        , 0.53571429, 0.        , 0.        , 1.        ,
        0.        , 0.66666667, 0.        , 1.        , 0.        ],
       [0.33333333, 0.82352941, 0.5       , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.39385936,
        0.25      , 0.25      , 1.        , 0

In [8]:
def data_generator(data,labels,batch_size) :
    class_data = [[],[]]
    for i,row in enumerate(data) :
        if labels[i] == 0 :
            class_data[0].append(list(row))
        else :
            class_data[1].append(list(row))
    while True :
        batch = []
        batch_label = []
        for _ in range(batch_size) :
            choice = 1 if np.random.random() > .5 else 0
            batch_label.append(choice)
            batch.append(class_data[choice][np.random.randint(len(class_data[choice]))])
        yield (np.array(batch),np.array(batch_label))     